In [17]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time
import os
import json
import sqlite3
from dotenv import load_dotenv
import requests
from datetime import datetime
import schedule

# Load the .env file
load_dotenv()

# Access the variables as environment variables
weibo_url = os.getenv('WEIBO_URL')
message_webhook_url = os.getenv('MESSAGE_WEBHOOK_URL')
status_webhook_url = os.getenv('STATUS_WEBHOOK_URL')

class WeiboScrapper:
    def __init__(self):
        # Setup driver
        # add headless
        options = Options()
        options.add_argument('--headless')
        self.driver = webdriver.Chrome(\
            service=Service(ChromeDriverManager().install())\
                ,options=options)
        # create a sqlite database to store id
        # change to mongodb later
        self.db = sqlite3.connect('weibo.db')
        self.cursor = self.db.cursor()
        self.cursor.execute('''CREATE TABLE IF NOT EXISTS weibo (id INTEGER PRIMARY KEY)''')
        self.db.commit()


    def start(self):
        # Run the scan immediately and then every 10 minutes
        self.scan()
        schedule.every(10).minutes.do(self.scan)

        # Run send_status immediately and then every hour
        schedule.every(1).hour.do(self.send_status)

        while True:
            schedule.run_pending()
            time.sleep(1)

        
    def get_weibo_content_once(self):
        try:
            self.driver.get(os.getenv('WEIBO_URL'))
            # Wait for the dynamic content to load
            time.sleep(5)
            self.driver.implicitly_wait(5)
            pre_tag = self.driver.find_element(By.TAG_NAME, 'pre')
            json_text = pre_tag.text
            self.driver.quit()
        except Exception as e:
            print(e)
            self.driver.quit()
            return None
        content = json.loads(json_text) # content is a dictionary
        return content['data']['list']
    
    def check_id(self,item):
        # if id is not in the database, return True
        # else return False
        weibo_item_id=item['id']
        self.cursor.execute('''SELECT * FROM weibo WHERE id=?''',(weibo_item_id,))
        if self.cursor.fetchone() is None:
            #write the id to the database
            self.cursor.execute('''INSERT INTO weibo (id) VALUES (?)''',(weibo_item_id,))
            self.db.commit()
            return True
        else:
            return False       

    def get_weibo_content_loop(self):
        i=0
        print('getting weibo content...')
        while True:
            content = self.get_weibo_content_once()
            if content:
                break   
            print('retrying...')
            time.sleep(5)
            i+=1
            print(i)
            if i>50:
                print('failed')
                return None
        return content

    def scan(self):
        content = self.get_weibo_content_loop()
        if content:
            for item in content:
                if self.check_id(item):
                    self.parse_item(item)
                    time.sleep(5)
        else:
            print('failed to get content')
            return None

    def parse_item(self,item):
        # parse item and store it in the database
        # send text_raw to discord
        # add separator to text_raw
        text_raw = item['text_raw']
        # use discord embed to display the content
        # "embed_color": 16738740
        
        message ={
            "embeds": [{
                "title": "塔菲の新微博喵~",
                "description": text_raw,
                "color": 16738740
            },
            ]
        }
        response = requests.post(message_webhook_url, json=message)
        return response.status_code

    def send_status():
        data = {"content": f"Script is running - {datetime.now().isoformat()}"}
        response = requests.post(status_webhook_url, json=data)
        return response.status_code

In [16]:
response.status_code

204

In [ ]:

    

scraper = WeiboScrapper()
content = scraper.get_weibo_content_loop()



In [2]:
# save content to a file
with open('weibo_content.json', 'w') as f:
    json.dump(content, f, indent=4, ensure_ascii=False)

In [3]:
content[0].keys()

dict_keys(['visible', 'created_at', 'id', 'idstr', 'mid', 'mblogid', 'user', 'can_edit', 'edit_count', 'text_raw', 'text', 'textLength', 'annotations', 'source', 'favorited', 'rid', 'pic_ids', 'geo', 'pic_num', 'pic_infos', 'is_paid', 'mblog_vip_type', 'title_source', 'reposts_count', 'comments_count', 'attitudes_count', 'attitudes_status', 'continue_tag', 'isLongText', 'mlevel', 'content_auth', 'is_show_bulletin', 'comment_manage_info', 'screen_name_suffix_new', 'share_repost_type', 'url_struct', 'isTop', 'mblogtype', 'showFeedRepost', 'showFeedComment', 'pictureViewerSign', 'showPictureViewer', 'rcList', 'region_name', 'customIcons'])

http://t.cn/A6pWaVOt 转发送10只雏草姬QQ装扮喵！💕
永雏塔菲× QQ联动来了喵！😎✌️
“永雏塔菲-兔兔”QQ装扮，即将于6月21日15:00在QQ平台上架喵！🐰
塔菲爱用的动态表情包😉、塔菲一直想要的粉红色ui喵🥰！塔菲梦寐以求的头像框👽、专属雏草姬编号💎、多套动态头图和聊天背景✨、获 
------------------
还真有p🙀怎么下海了喵
------------------
睡晚了呼呼呼喵💤
------------------
7.15～7.18在广州萤火虫萤之町Y01喵！👻
会一边看摊子一边直播的喵！📺
来taffy摊位玩小游戏白领taffy周边！🎁线下场贩限定新周边首发上线！😻超多互动活动详见视频喵！✨
7.15 14:30～15:30taffy还有taffy在主舞台表演和现场观众小游戏互动的环节喵！🎤🎮
#永雏塔菲[超话]#  
------------------
这是能在国产剧里看到的吗🙀？！？！//@小坏蛋新的一年也要继续国见命:[哆啦A梦害怕]点开前我还在想能tm有多大尺度，点开后：是我太天真了//@哇水野爱:点了链接发现广告不一般大概意识到了有多大尺度，结果想不到比脑测的更大.jpg//@性感村村长:http://t.cn/A6puKVKM  实测很吊
------------------
冰激凌🍦好好吃嗯嗯喵🥵夹总不要再夹了呜喵喵🤤
------------------
萌喵！饿喵！苹果糖！馋喵！🤤
------------------
下完来喵👻 
------------------
家暴女每日练习中！👊
#永雏塔菲[超话]# 
------------------
打喵拳！✊😚✌️
------------------
为爱提问的雏草姬准备的完全匿名提问箱喵！✍️http://t.cn/A6p34GOO
不定时直播回答喵！👻
taffy还是习惯用这个喵！！操作很简单的看塔叠亲截亲打的图喵！👀
#永雏塔菲[超话]# 
------------------


In [21]:
content[0]['pic_infos']

{'008jCdW0gy1hf4jwowf7vg30ku0h9dwc': {'thumbnail': {'url': 'https://wx2.sinaimg.cn/wap180/008jCdW0gy1hf4jwowf7vg30ku0h9dwc.gif',
   'width': 180,
   'height': 148,
   'cut_type': 1,
   'type': ''},
  'bmiddle': {'url': 'https://wx2.sinaimg.cn/wap360/008jCdW0gy1hf4jwowf7vg30ku0h9dwc.gif',
   'width': 360,
   'height': 297,
   'cut_type': 1,
   'type': ''},
  'large': {'url': 'https://wx2.sinaimg.cn/sti960/008jCdW0gy1hf4jwowf7vg30ku0h9dwc.gif',
   'width': 750,
   'height': 621,
   'cut_type': 1,
   'type': ''},
  'original': {'url': 'https://wx2.sinaimg.cn/orj1080/008jCdW0gy1hf4jwowf7vg30ku0h9dwc.gif',
   'width': 750,
   'height': 621,
   'cut_type': 1,
   'type': ''},
  'largest': {'url': 'https://wx2.sinaimg.cn/large/008jCdW0gy1hf4jwowf7vg30ku0h9dwc.gif',
   'width': 750,
   'height': 621,
   'cut_type': 1,
   'type': ''},
  'mw2000': {'url': 'https://wx2.sinaimg.cn/mw2000/008jCdW0gy1hf4jwowf7vg30ku0h9dwc.gif',
   'width': 750,
   'height': 621,
   'cut_type': 1,
   'type': ''},
  'o

In [19]:
content[0]['']

{'visible': {'type': 0, 'list_id': 0},
 'created_at': 'Tue Jun 20 12:01:01 +0800 2023',
 'id': 4914676545359937,
 'idstr': '4914676545359937',
 'mid': '4914676545359937',
 'mblogid': 'N69NQmumB',
 'user': {'id': 7618923072,
  'idstr': '7618923072',
  'pc_new': 7,
  'screen_name': '永雏塔菲',
  'profile_image_url': 'https://tvax4.sinaimg.cn/crop.0.0.1080.1080.50/008jCdW0ly8hfdxzryxz7j30u00u0gpc.jpg?KID=imgbed,tva&Expires=1688320522&ssig=aZeHk7TMYQ',
  'profile_url': '/u/7618923072',
  'verified': True,
  'verified_type': 0,
  'domain': 'acetaffy',
  'weihao': '',
  'verified_type_ext': 1,
  'avatar_large': 'https://tvax4.sinaimg.cn/crop.0.0.1080.1080.180/008jCdW0ly8hfdxzryxz7j30u00u0gpc.jpg?KID=imgbed,tva&Expires=1688320522&ssig=42vYRwga4j',
  'avatar_hd': 'https://tvax4.sinaimg.cn/crop.0.0.1080.1080.1024/008jCdW0ly8hfdxzryxz7j30u00u0gpc.jpg?KID=imgbed,tva&Expires=1688320522&ssig=0rsc1lAgUN',
  'follow_me': False,
  'following': False,
  'mbrank': 5,
  'mbtype': 12,
  'v_plus': 0,
  'planet